In [2]:
# 라이브러리 호출
import webbrowser
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.alert import Alert 

# 원티드 내 직군 분류 크롤링
- 실행하지 않아도 무방

In [3]:
# 브라우저 생성 
browser = webdriver.Chrome()

# 입력 한 주소로 브라우저 이동
browser.get("https://www.wanted.co.kr/jobsfeed")

time.sleep(2)
# 알람 팝업시 확인버튼 누르기
# Alert(browser).accept() 

# 채용 메뉴 클릭
browser.find_element(By.CSS_SELECTOR, "a.Menu_jobListLink__GYHAI").click()
browser.find_element(By.CSS_SELECTOR, "span.MoreButton_MoreButton__2UmZ8").click()


# 채용 메뉴에 있는 직군 목록을 리스트 형식으로 저장하기
items = browser.find_elements(By.CSS_SELECTOR, "a.JobGroupItem_JobGroupItem__xXzAi")

job_list = []

for item in items:
    job_list.append(item.text)

print(job_list)
# ['개발', '경영·비즈니스', '마케팅·광고', '디자인', '영업', '고객서비스·리테일', '미디어', '엔지니어링·설계', '게임 제작', 'HR', '금융', '제조·생산', '물류·무역', '', '', '', '', '', '']
# 물류·무역 까지는 정상적으로 크롤링 됨 -> 이후의 값들은 ''로 불러와짐
# 해당 카테고리의 스크롤을 내려야 할거같은데 못찾겠어서 skip


['개발', '경영·비즈니스', '마케팅·광고', '디자인', '영업', '고객서비스·리테일', '미디어', '엔지니어링·설계', '게임 제작', 'HR', '금융', '제조·생산', '물류·무역', '', '', '', '', '', '']


# 데이터 직군 공고 크롤링
- 머신러닝 엔지니어, 데이터 엔지니어, 데이터 사이언티스트, 빅데이터 엔지니어

In [4]:
# 데이터 직군 채용공고 모든 url 추출하기

browser = webdriver.Chrome()
browser.get("https://www.wanted.co.kr/wdlist/518?country=kr&job_sort=job.latest_order&years=-1&selected=1634&selected=655&selected=1024&selected=1025&locations=all")

time.sleep(2)

before_h = browser.execute_script("return window.scrollY") # execute_script : 자바명령어 실행

# 원티드 무한스크롤 끝까지 내리기

while True: #무한반복
    # 맨 아래로 스크롤을 내린다.
    browser.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
    # 스크롤 사이 페이지 로딩시간고려 약간의 텀 주기
    time.sleep(3)

    # 스크롤 후 높이
    after_h = browser.execute_script("return window.scrollY")
    
    # 끝까지 내려간 경우 stop
    if after_h == before_h:
        break
    before_h = after_h


# 스크롤 끝까지 내린 후 모든 공고의 url 추출
urls = browser.find_elements(By.CSS_SELECTOR, ".Card_className__u5rsb > a")

url_list = []
for url in urls:
    url = url.get_attribute("href")
    url_list.append(url)

print(url_list)
print(len(url_list))

# 총 738개 url 추출 완료

['https://www.wanted.co.kr/wd/182774', 'https://www.wanted.co.kr/wd/182758', 'https://www.wanted.co.kr/wd/182633', 'https://www.wanted.co.kr/wd/182617', 'https://www.wanted.co.kr/wd/182615', 'https://www.wanted.co.kr/wd/182566', 'https://www.wanted.co.kr/wd/182506', 'https://www.wanted.co.kr/wd/182487', 'https://www.wanted.co.kr/wd/182486', 'https://www.wanted.co.kr/wd/182428', 'https://www.wanted.co.kr/wd/182419', 'https://www.wanted.co.kr/wd/182407', 'https://www.wanted.co.kr/wd/182404', 'https://www.wanted.co.kr/wd/182362', 'https://www.wanted.co.kr/wd/182293', 'https://www.wanted.co.kr/wd/182251', 'https://www.wanted.co.kr/wd/182239', 'https://www.wanted.co.kr/wd/182235', 'https://www.wanted.co.kr/wd/182232', 'https://www.wanted.co.kr/wd/182213', 'https://www.wanted.co.kr/wd/182178', 'https://www.wanted.co.kr/wd/182134', 'https://www.wanted.co.kr/wd/182130', 'https://www.wanted.co.kr/wd/182124', 'https://www.wanted.co.kr/wd/182075', 'https://www.wanted.co.kr/wd/181976', 'https://ww

# Url을 100개 단위로 나누기

In [5]:
url_list_1 = url_list[:100]
url_list_2 = url_list[100:200]
url_list_3 = url_list[200:300]
url_list_4 = url_list[300:400]
url_list_5 = url_list[400:500]
url_list_6 = url_list[500:600]
url_list_7 = url_list[600:700]
url_list_8 = url_list[700:]

# 함수생성

In [7]:
def prefer_list(url_list): # 채용공고의 url list입력 시 우대사항 정보 및 크롤링 실패한 url 주소 반환
    success = 0 # 성공횟수
    fail = 0 # 실패횟수
    prefer = []
    fail_url = [] # 크롤링에 실패 한 url 저장

    for url in url_list:
        browser = webdriver.Chrome()
        browser.get(url) # 크롤링 한 채용공고의 url    
                
        try :
            details = browser.find_elements(By.CSS_SELECTOR, "div.JobContent_descriptionWrapper__SM4UD p > span")
            # JobContent_descriptionWrapper__SM4UD 클래스의 하위 p > 하위 span태그 추출
            prefer.append(details[3].text) # span태그의 3번째가 우대사항. -> 3번째면 prefer 리스트에 저장
            browser.quit()
            success += 1
            print("success_count : ", success)

        except :
            fail += 1
            fail_url.append(url)
            browser.quit()
            print("fail_count : ", fail)
    print(prefer)
    return prefer, fail_url

# 함수 호출 및 csv저장

In [8]:
prefer1, fail_url1 = prefer_list(url_list_1)
pd.DataFrame(prefer1).to_csv("prefer1.csv")
pd.DataFrame(fail_url1).to_csv("fail_url1.csv")

prefer2, fail_url2 = prefer_list(url_list_2)
pd.DataFrame(prefer2).to_csv("prefer2.csv")
pd.DataFrame(fail_url2).to_csv("fail_url2.csv")

prefer3, fail_url3 = prefer_list(url_list_3)
pd.DataFrame(prefer3).to_csv("prefer3.csv")
pd.DataFrame(fail_url3).to_csv("fail_url3.csv")


prefer4, fail_url4 = prefer_list(url_list_4)
pd.DataFrame(prefer4).to_csv("prefer4.csv")
pd.DataFrame(fail_url4).to_csv("fail_url4.csv")


prefer5, fail_url5 = prefer_list(url_list_5)
pd.DataFrame(prefer5).to_csv("prefer5.csv")
pd.DataFrame(fail_url5).to_csv("fail_url5.csv")


prefer6, fail_url6 = prefer_list(url_list_6)
pd.DataFrame(prefer6).to_csv("prefer6.csv")
pd.DataFrame(fail_url6).to_csv("fail_url6.csv")


prefer7, fail_url7 = prefer_list(url_list_7)
pd.DataFrame(prefer7).to_csv("prefer7.csv")
pd.DataFrame(fail_url7).to_csv("fail_url7.csv")


prefer8, fail_url8 = prefer_list(url_list_8)
pd.DataFrame(prefer8).to_csv("prefer8.csv")
pd.DataFrame(fail_url8).to_csv("fail_url8.csv")


success_count :  1
success_count :  2
success_count :  3
success_count :  4
success_count :  5
success_count :  6
success_count :  7
success_count :  8
success_count :  9
success_count :  10
success_count :  11
success_count :  12
success_count :  13
success_count :  14
success_count :  15
success_count :  16
success_count :  17
success_count :  18
success_count :  19
success_count :  20
success_count :  21
success_count :  22
success_count :  23
success_count :  24
success_count :  25
success_count :  26
success_count :  27
success_count :  28
success_count :  29
success_count :  30
success_count :  31
success_count :  32
success_count :  33
success_count :  34
success_count :  35
success_count :  36
success_count :  37
success_count :  38
success_count :  39
success_count :  40
success_count :  41
success_count :  42
success_count :  43
success_count :  44
success_count :  45
success_count :  46
success_count :  47
success_count :  48
success_count :  49
success_count :  50
success_c